# This notebook is a package of "Greedy Forward Selection"

In [0]:
# -*- coding: utf-8 -*-

## Abstract of this package in Japanese heavy version  
本手法は特徴量の組み合わせを変えてモデル(xgboost)に学習させることを繰り返し、その精度をもとに重要な変数を探索していきます。手順は以下。  
1. 使用する特徴量の集合を空から始める（この集合をMとする）  
2. 候補となる特徴量のそれぞれについてMに加えた場合のスコアを計算する  
3. 最もスコアが改善した特徴量をMに加える
4. ３に採用された特徴量を候補から外し、2.3.をスコアの改善が止まるまで繰り返す  

## Data Preparation

In [0]:
import numpy as np
import pandas as pd

train = pd.read_csv('../content/sample_data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../content/sample_data/test_preprocessed.csv')

In [6]:
train.head()

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,medical_info_b2,medical_info_b3,medical_info_c1,medical_info_c2,medical_keyword_1,medical_keyword_2,medical_keyword_3,medical_keyword_4,medical_keyword_5,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,target
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,2,1,1.0,NaN,1,0,0,0,0,1,0,1,0,0,2015,2,3,24182,0
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,1,7,NaN,NaN,0,0,0,0,0,0,1,1,0,0,2015,5,9,24185,0
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,3,11,2.0,NaN,1,1,1,0,0,1,0,1,0,0,2016,2,13,24194,1
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,0,9,2.0,16.167065,1,0,0,0,0,0,0,1,0,0,2015,7,6,24187,0
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,0,9,2.0,NaN,0,1,1,0,0,0,1,1,1,0,2016,9,17,24201,1


In [7]:
train.shape, test_x.shape

((10000, 29), (10000, 28))

In [0]:
# devide dataset to training and validarion
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [0]:
# define evaluate function using xgboost
import xgboost as xgb
from sklearn.metrics import log_loss

def evaluate(features):
    dtrain = xgb.DMatrix(tr_x[features], label=tr_y)
    dvalid = xgb.DMatrix(va_x[features], label=va_y)
    params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}  #if you want to use regression and square error, you should chose objective:"reg:squarederror"
    num_round = 1000 
    early_stopping_rounds = 10
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round,
                      evals=watchlist, early_stopping_rounds=early_stopping_rounds,
                      verbose_eval=0)
    va_pred = model.predict(dvalid)
    score = log_loss(va_y, va_pred)

    return score

## Greedy Forward Selection

In [10]:
best_score = 9999.0
selected = set([])

print('start greedy forward selection')

while True:

    if len(selected) == len(train_x.columns):
        # all features selected and end
        break

    scores = []
    for feature in train_x.columns:
        if feature not in selected:
            fs = list(selected) + [feature]
            score = evaluate(fs)
            scores.append((feature, score))

    # minimum score is the best in this example. If you use other scoring index, you need rewrite below.
    b_feature, b_score = sorted(scores, key=lambda tpl: tpl[1])[0]
    if b_score < best_score:
        selected.add(b_feature)
        best_score = b_score
        print(f'selected:{b_feature}')
        print(f'score:{b_score}')
    else:
        # we can't improve score any more, so finish
        break

print(f'selected features: {selected}')

start greedy forward selection


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

selected:medical_info_a1
score:0.4669966796517372


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

selected:weight
score:0.4380964568354189


...

selected:medical_keyword_6
score:0.20510625407898186


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

selected:medical_keyword_7
score:0.20498131129573158


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

selected features: {'medical_info_a1', 'height', 'medical_keyword_1', 'medical_keyword_5', 'age', 'weight', 'medical_keyword_4', 'medical_keyword_6', 'medical_keyword_7', 'medical_keyword_2', 'medical_keyword_3', 'product', 'medical_info_a2', 'amount', 'medical_info_a3', 'medical_info_c1'}


In [12]:
print(selected, len(selected))
#after feature selection reduce 12 features.

{'medical_info_a1', 'height', 'medical_keyword_1', 'medical_keyword_5', 'age', 'weight', 'medical_keyword_4', 'medical_keyword_6', 'medical_keyword_7', 'medical_keyword_2', 'medical_keyword_3', 'product', 'medical_info_a2', 'amount', 'medical_info_a3', 'medical_info_c1'} 16


## Abstract of this package in Japanese lite version  
上記のGreedy forward selectionは特徴量が増えると計算量が特徴量の数の２乗に比例する。単純化し、軽くした（特徴量の数に比例）ものの手順は以下。  
1. 使用する特徴量の集合を空から始める（この集合をMとする）  
2. 特徴量の候補を有望な順ｏｒランダムに並べる  
3. 次の特徴量を加えてスコア（評価指標）が改善すればMに加える（そうならない場合は加えない）  
4. ３をすべての候補について繰り返す　　  

In [13]:
best_score = 9999.0
candidates = np.random.RandomState(71).permutation(train_x.columns)
selected = set([])

print('start simple selection')
for feature in candidates:
    fs = list(selected) + [feature]
    score = evaluate(fs)

    if score < best_score:
        selected.add(feature)
        best_score = score
        print(f'selected:{feature}')
        print(f'score:{score}')

print(f'selected features: {selected}')

start simple selection
selected:product
score:0.4858118335008621


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


selected:weight
score:0.46987540227174757


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


selected:height
score:0.4169032759234309


...

selected:year
score:0.2628804816475138


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


selected:medical_info_a3
score:0.2588787440238986


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


selected features: {'medical_info_a1', 'height', 'medical_keyword_1', 'medical_keyword_5', 'age', 'year', 'weight', 'medical_keyword_4', 'medical_info_b1', 'medical_keyword_2', 'medical_keyword_3', 'product', 'medical_info_a3'}


In [14]:
print(selected, len(selected))
#after feature selection reduce 15 features.

{'medical_info_a1', 'height', 'medical_keyword_1', 'medical_keyword_5', 'age', 'year', 'weight', 'medical_keyword_4', 'medical_info_b1', 'medical_keyword_2', 'medical_keyword_3', 'product', 'medical_info_a3'} 13
